### AutoGen Advanced

This file containes a simple project to learn more on AutoGen AgentChat. The concepts covered in this file consists of:

1. Multi-modal conversation
2. Structured Outputs
3. Using LangChain tools
4. Teams
5. Model Context Protocol(MCP)

In [ ]:
# Importing required libraries
from io import BytesIO
import requests
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from typing import Literal
from PIL import Image
from dotenv import load_dotenv

# Importing necessary libraries from autogen
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken


In [ ]:
load_dotenv(override=True)

#### A multi-modal conversation
The model has an image as an input and the agent describes the image.

In [ ]:
url = "https://edwarddonner.com/wp-content/uploads/2024/10/from-software-engineer-to-AI-DS.jpeg"

pil_image = Image.open(BytesIO(requests.get(url).content))
img = AGImage(pil_image)
img

In [ ]:
multi_modal_message = MultiModalMessage(content=["Describe the content of this image in detail", img], source="User")

In [ ]:
# Initializing the gemini client and image describer agent using the client
gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash")

image_describer = AssistantAgent(
    name="image_describer_agent",
    model_client=gemini_client,
    system_message="You are good at describing images"
)

#### Structured Outputs!

Autogen AgentChat makes it easy to use structured outputs.

In [ ]:
class ImageDescription(BaseModel):
    scene: str = Field(description="Briefly describe the overall scene of the image")
    message: str = Field(description="The point that the image is trying to convey")
    style: str = Field(description="The artistic stye of image")
    orientation: Literal["portrait", "landscape", "square"] = Field(description="The orientation of the image")

In [ ]:
# Image describer agent and calling the agent
image_describer = AssistantAgent(
    name="image_describer_agent",
    model_client=gemini_client,
    system_message="You are good at describing images in detail",
    output_content_type=ImageDescription
)

response = await image_describer.on_messages([multi_modal_message], cancellation_token=CancellationToken())
reply = response.chat_message.content
reply


In [ ]:
import textwrap
print(f"Scene:\n{textwrap.fill(reply.scene)}\n")
print(f"Message:\n{textwrap.fill(reply.message)}\n")
print(f"Style:\n{textwrap.fill(reply.style)}\n")
print(f"Orientation:\n{textwrap.fill(reply.orientation)}\n")

#### Using LangChain tools from AutoGen
An advantage of using AutoGen AgentChat is that tools in Langchain, which are updated by the community are available for use by simply calling `LangChainToolAdapter`.

In [ ]:
# AutoGen's wrapper:
import os
os.makedirs("output", exist_ok=True)
from unittest import result
from autogen_ext.tools.langchain import LangChainToolAdapter

# LangChain tools:

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain.agents import Tool


prompt = """Your task is to find a one-way non-stop flight from Nairobi to Las Vegas, through Dubai in December 2025.
First search online for promising deals.
Next, write all the deals to a file called flights.md with full details.
Finally, select the one you think is best and reply with a short summary.
Reply with the selected flight only, and only after you have written the details to the file."""

# Internet Search
serper = GoogleSerperAPIWrapper()

langchain_serper_tool = Tool(
    name="internet_search_tool",
    func=serper.run,
    description="Useful for running internet searches"
)
autogen_serper = LangChainToolAdapter(langchain_serper_tool)
autogen_tools = [autogen_serper]

# Langchain file management tool
langchain_file_management_tools = FileManagementToolkit(root_dir="output").get_tools()
for tool in langchain_file_management_tools:
    autogen_tools.append(LangChainToolAdapter(tool))

for tool in autogen_tools:
    print(tool.name, tool.description)

internet_search_agent = AssistantAgent(
    name="internet_search_agent",
    model_client=gemini_client,
    tools=autogen_tools,
    reflect_on_tool_use=True
)

message = TextMessage(content=prompt, source="user")
result = await internet_search_agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))

In [ ]:
# Now we need to call the internet_search_agent again to write the file

message = TextMessage(content="OK proceed", source="user")

result = await internet_search_agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))

#### Team in AutoGen
A [team](https://microsoft.github.io/autogen/stable//user-guide/agentchat-user-guide/tutorial/teams.html) is a group of agents that work together to achieve a common goal.

AgentChat supports several team presets:

- `RoundRobinGroupChat`: A team that runs a group chat with participants taking turns in a round-robin fashion (covered on this page). Tutorial

- `SelectorGroupChat`: A team that selects the next speaker using a ChatCompletion model after each message. Tutorial

- `MagenticOneGroupChat`: A generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. Tutorial

- `Swarm`: A team that uses `HandoffMessage` to signal transitions between agents

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import  TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool

serper = GoogleSerperAPIWrapper()
internet_search_tool =Tool(
    name="internet_search_tool", 
    func=serper.run, 
    description="Useful for running internet searches")
autogen_serper = LangChainToolAdapter(internet_search_tool)

prompt = """Find a one-way non-stop flight from Nairobi to Las Vegas in December 2025."""

# Create primary agent
primary_agent = AssistantAgent(
    "primary",
    model_client=gemini_client,
    tools=[autogen_serper],
    system_message="You are a helpful AI research assistant who looks for promising deals on flights. Incorporate any feedback you receive.",
)

# Create evaluator agent
evaluation_agent = AssistantAgent(
    "evaluator",
    model_client=gemini_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' when your feedback is addressed.",
)

# Define a termination condition that stops the task
text_termination = TextMentionTermination("APPROVE")

#Team definition
team = RoundRobinGroupChat([primary_agent, evaluation_agent], termination_condition=text_termination, max_turns=10)


In [ ]:
result = await team.run(task=prompt)
for message in result.messages:
    print(f"{message.source}:\n{message.content}\n\n")

#### MCP Introdiction

Our first look at the Model Context Protocol from Anthropic -

Autogen makes it easy to use MCP tools, just like LangChain tools.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# Get the fetch tool from mcp-server-fetch.
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"], read_timeout_seconds=30)
fetcher = await mcp_server_tools(fetch_mcp_server)

# Create an agent that can use the fetch tool.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="fetcher", model_client=model_client, tools=fetcher, reflect_on_tool_use=True)  # type: ignore

# Let the agent fetch the content of a URL and summarize it.
result = await agent.run(task="Review edwarddonner.com and summarize what you learn. Reply in Markdown.")
display(Markdown(result.messages[-1].content))